## Ingest Qualifying Folder

In [0]:
dbutils.widgets.text('p_data_source', '')
v_data_source = dbutils.widgets.get('p_data_source')

In [0]:
dbutils.widgets.text('p_file_date', '2021-03-28')
v_file_date = dbutils.widgets.get('p_file_date')

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Step 1: Read Qualifying file with manual schema

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

qualifying_schema = StructType([StructField('qualifyId', IntegerType(), False),
                               StructField('raceId', IntegerType(), False),
                               StructField('driverId', IntegerType(), False),
                               StructField('constructorId', IntegerType(), False),
                               StructField('number', IntegerType(), True),
                               StructField('position', IntegerType(), True),
                               StructField('q1', StringType(), True),
                               StructField('q2', StringType(), True),
                               StructField('q3', StringType(), True)])

In [0]:
qualifying_df = spark.read.table('formula1.bronze.qualifying')

#### Step 2: Rename columns

In [0]:
qualifying_renamed_df= qualifying_df.withColumnRenamed('qualifyId', 'qualify_id')\
                                    .withColumnRenamed('raceId', 'race_id')\
                                    .withColumnRenamed('driverId', 'driver_id')\
                                    .withColumnRenamed('constructorId', 'constructor_id')


#### Step 3: Add Ingestion date

In [0]:
from pyspark.sql.functions import lit

qualifying_final_df = add_ingestion_date(qualifying_renamed_df.withColumn('data_source', lit(v_data_source)))


#### Step 4: Save data to data lake in parquet

In [0]:
#incremental_load(qualifying_final_df, 'f1_processed', 'qualifying', 'race_id')


In [0]:
folder_path = silver_folder_path + '/qualifying'
merge_condition = "tgt.qualify_id = src.qualify_id AND tgt.race_id = src.race_id"
partition_column = 'race_id'
merge_delta_data(qualifying_final_df, 'formula1', 'silver', 'qualifying', merge_condition, partition_column)

In [0]:
dbutils.notebook.exit("Success")